## Import

In [10]:
import glob
import torch
from torch.utils import data
from PIL import Image
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from ahrs.filters import Madgwick
from ahrs.common.orientation import q2euler
from PIL import Image, ImageDraw
import os

#通过创建data.Dataset子类Mydataset来创建输入
class Mydataset(data.Dataset):
# 类初始化
    def __init__(self, root):
        self.imgs_path = root
# 进行切片
    def __getitem__(self, index):
        img_path = self.imgs_path[index]
        return img_path
# 返回长度
    def __len__(self):
        return len(self.imgs_path)


## Go through every image in the folder

In [24]:
#使用glob方法来获取数据图片的所有路径
all_imgs_path = glob.glob(r'/home/lycho-_-/文档/AML_LAB/row_data/signature_zeyang/*.csv')#数据文件夹路径，根据实际情况更改！
new_img_dir = '/home/lycho-_-/文档/AML_LAB/img_data'  
new_imu_dir = '/home/lycho-_-/文档/AML_LAB/imu_data'  
#循环遍历输出列表中的每个元素，显示出每个图片的路径


#利用自定义类Mydataset创建对象weather_dataset
signature_dataset = Mydataset(all_imgs_path)
print(len(signature_dataset)) #返回文件夹中图片总个数
print(signature_dataset[12:15])#切片，显示第12至第十五张图片的路径
sinature_datalodaer = torch.utils.data.DataLoader(signature_dataset, batch_size=5) #每次迭代时返回五个数据
#print(next(iter(sinature_datalodaer)))




5
[]


## Transform IMU

In [15]:
species = ['False','True']
species_to_id = dict((c, i) for i, c in enumerate(species))
#print(species_to_id)
id_to_species = dict((v, k) for k, v in species_to_id.items())
#print(id_to_species)
all_labels = []
#对所有图片路径进行迭代
for imu in all_imgs_path:
    # 区分出每个img，应该属于什么类别
    for i, c in enumerate(species):
        if c in imu:
            all_labels.append(i)
print(all_labels) #得到所有标签 


for var in all_imgs_path:
    data = pd.read_csv(var)
    x_coords=data.values[0::,1]
    y_coords=data.values[0::,2]
    pressure=data.values[0::,3]
    acc_data1 = data[['accel_x', 'accel_y', 'accel_z']].to_numpy()
    gyro_data1 = data[['gyro_x', 'gyro_y', 'gyro_z']].to_numpy()
    # 初始化Madgwick过滤器，这里假设数据采样率为50Hz
    delta_t = 1/50
    madgwick = Madgwick(delta_t=delta_t)
    q = [1, 0, 0, 0]
    # 使用加速度计和陀螺仪数据处理IMU数据，估算姿态
    # 不需要显式指定初始四元数，Madgwick过滤器会自动处理
    quaternions = np.array([madgwick.updateIMU(q=q,acc=acc_data1[i], gyr=gyro_data1[i]) for i in range(len(acc_data1))])

    # 将四元数转换为欧拉角，以便更容易解释
    euler_angles = np.array([q2euler(q) for q in quaternions])
    euler_angles_df = pd.DataFrame(euler_angles, columns=['roll', 'pitch', 'yaw'])

    combined_data = pd.concat([data[['x', 'y', 'pressure']], euler_angles_df], axis=1)

      
    base_name = os.path.basename(var)  # 获取原始文件的基本名称
    new_imu_name = os.path.splitext(base_name)[0] + 'ruiqi.csv'  # 更改文件扩展名为.png
    new_imu_path = os.path.join(new_imu_dir, new_imu_name)  # 创建新的文件路径
    print(new_imu_path)
    # 保存到新的CSV文件
    combined_data.to_csv(new_imu_path, index=False)


    combined_data.head()


[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
/home/lycho-_-/文档/AML_LAB/imu_data/True_1ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/True_2ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/True_3ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/True_4ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/True_5ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/False_1ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/False_2ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/False_3ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/False_4ruiqi.csv
/home/lycho-_-/文档/AML_LAB/imu_data/False_5ruiqi.csv


## Transform IMG

In [26]:
species = ['False','True']
species_to_id = dict((c, i) for i, c in enumerate(species))
id_to_species = dict((v, k) for k, v in species_to_id.items())
all_labels = []
#对所有图片路径进行迭代
for imu in all_imgs_path:
    # 区分出每个img，应该属于什么类别
    for i, c in enumerate(species):
        if c in imu:
            all_labels.append(i)



for var in all_imgs_path:
    print(var)
    data = pd.read_csv(var)
    x_coords=data.values[0::,1]
    y_coords=data.values[0::,2]
    pressure=data.values[0::,3]
    pressure = pressure-np.mean(pressure)
# 创建一个白色背景的图像
    img = Image.new('RGB', (240, 320), color='white')
    draw = ImageDraw.Draw(img)


    max_pressure = np.max(pressure)

    for x, y, p in zip(x_coords, y_coords, pressure):
        #print(p)
        if p > 0:  # 只绘制压力大于0的点
        
            color = (int(p / max_pressure *255), 0, 0)  # 根据压力值生成颜色
            draw.point((x, y), fill=color)




    img = img.rotate(90)            
    base_name = os.path.basename(var)  # 获取原始文件的基本名称
    new_img_name = os.path.splitext(base_name)[0] + 'zeyang.png'  # 更改文件扩展名为.png
    new_img_path = os.path.join(new_img_dir, new_img_name)  # 创建新的文件路径
    
    img.save(new_img_path)  # 保存图像到新的文件路径
    # 保存到新的CSV文件

/home/lycho-_-/文档/AML_LAB/row_data/signature_zeyang/True_1.csv
/home/lycho-_-/文档/AML_LAB/row_data/signature_zeyang/True_2.csv
/home/lycho-_-/文档/AML_LAB/row_data/signature_zeyang/True_3.csv
/home/lycho-_-/文档/AML_LAB/row_data/signature_zeyang/True_4.csv
/home/lycho-_-/文档/AML_LAB/row_data/signature_zeyang/True_5.csv
